In [1]:
#Install Java:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
#Download and Install Spark: Use the following commands to download and set up Spark:

!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz

In [7]:
#Set Environment Variables: Set the necessary environment variables for Spark and Java:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [41]:
#Install PySpark: Install PySpark using pip:
!pip install -q pyspark

In [42]:
#Start Spark Session:
!pip install findspark

import findspark
findspark.init()

# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [43]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'),header=True, ignoreLeadingWhiteSpace=True, inferSchema=True, quote="\"" , escape="\"", )
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [44]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [45]:
print("Total Record in DataFrame : ", df.count())
df.printSchema()

Total Record in DataFrame :  33287
root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [46]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
  SELECT YEAR(date), ROUND(AVG(price),2) as avg_price_4BR
  FROM home_sales
  WHERE bedrooms==4
  GROUP BY YEAR(date)
  ORDER By YEAR(date) desc
"""
avg_price_4_BR = spark.sql(query).show()

+----------+-------------+
|year(date)|avg_price_4BR|
+----------+-------------+
|      2022|    296363.88|
|      2021|    301819.44|
|      2020|    298353.78|
|      2019|     300263.7|
+----------+-------------+



In [47]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
  SELECT date_built, ROUND(AVG(price),2) as avg_price_3BR_3Bath
  FROM home_sales
  WHERE bedrooms==3 AND bathrooms==3
  GROUP BY date_built
  ORDER By date_built desc
"""
avg_price_3BR_3Bath = spark.sql(query).show()

+----------+-------------------+
|date_built|avg_price_3BR_3Bath|
+----------+-------------------+
|      2017|          292676.79|
|      2016|          290555.07|
|      2015|           288770.3|
|      2014|          290852.27|
|      2013|          295962.27|
|      2012|          293683.19|
|      2011|          291117.47|
|      2010|          292859.62|
+----------+-------------------+



In [48]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
  SELECT date_built, ROUND(AVG(price),2) as avg_price_3BR_3Bath_2floor_2Ksqft
  FROM home_sales
  WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living >= 2000
  GROUP BY date_built
  ORDER By date_built desc
"""
avg_price_3BR_3Bath_2floor_2Ksqft = spark.sql(query).show()


+----------+---------------------------------+
|date_built|avg_price_3BR_3Bath_2floor_2Ksqft|
+----------+---------------------------------+
|      2017|                        280317.58|
|      2016|                         293965.1|
|      2015|                        297609.97|
|      2014|                        298264.72|
|      2013|                        303676.79|
|      2012|                        307539.97|
|      2011|                        276553.81|
|      2010|                        285010.22|
+----------+---------------------------------+



In [49]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

import time
start_time = time.time()

query = """
  SELECT view, ROUND(AVG(price),2) as avg_price_by_view
  FROM home_sales
  GROUP BY view
  HAVING ROUND(AVG(price),2) >= 350000
  ORDER BY view DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------+
|view|avg_price_by_view|
+----+-----------------+
| 100|        1026669.5|
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 0.3898930549621582 seconds ---


In [50]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [51]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [52]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
  SELECT view, ROUND(AVG(price),2) as avg_price_by_view
  FROM home_sales
  GROUP BY view
  HAVING ROUND(AVG(price),2) >= 350000
  ORDER BY view DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------+
|view|avg_price_by_view|
+----+-----------------+
| 100|        1026669.5|
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 0.25015926361083984 seconds ---


In [53]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("partitioned_home_sales")

In [54]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('partitioned_home_sales')

In [55]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('temp_home_parquet')

In [56]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
  SELECT view, ROUND(AVG(price),2) as avg_price_by_view
  FROM temp_home_parquet
  GROUP BY view
  HAVING ROUND(AVG(price),2) >= 350000
  ORDER BY view DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------+
|view|avg_price_by_view|
+----+-----------------+
| 100|        1026669.5|
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 0.3461942672729492 seconds ---


In [57]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [58]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False